In [1]:
import pickle
import os
import re
import numpy as np
from gensim.models import word2vec
import logging
import pandas as pd

In [2]:
import sys
sys.path.insert(0, '../Core-scripts/')

from parse_and_prepare import ProteinProteinInteractionClassifier as ppi
import file_readers as fr
import prediction as pred

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

/usr/local/lib/python3.5/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [7]:
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [4]:
yeast_strict_real = pickle.load(open('../../Results/Yeast/yeast_mentions_strict_real.pkl', 'rb'))
yeast_gen_real = pickle.load(open('../../Results/Yeast/yeast_mentions_gen_real.pkl', 'rb'))
yeast_be_real = pickle.load(open('../../Results/Yeast/yeast_mentions_be_real.pkl', 'rb'))
random_seeds = [144, 235, 905, 2895, 3462, 4225, 5056, 5192, 7751, 7813]

In [10]:
for seed in random_seeds:
    real_tr_te_name = 'Yeast/train_test/yeast_tr_te_split_' + str(seed)
    train_data, b, c, d = pred.manual_train_test_split(yeast_strict_real, real_tr_te_name, random_state=seed ,test_set_prop=0.1)

In [22]:
yeast_w2v_model_strict = pred.make_w2v_model(yeast_strict_real, 'Yeast/yeast_strict', [300, 5, 4, 6, 0.001])
yeast_w2v_model_gen = pred.make_w2v_model(yeast_gen_real, 'Yeast/yeast_gen', [300, 5, 4, 6, 0.001])
yeast_w2v_model_be = pred.make_w2v_model(yeast_be_real, 'Yeast/yeast_be', [300, 5, 4, 6, 0.001])

2017-05-16 12:31:58,774 : INFO : collecting all words and their counts
2017-05-16 12:31:58,774 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-16 12:31:58,816 : INFO : collected 11431 word types from a corpus of 188505 raw words and 7145 sentences
2017-05-16 12:31:58,817 : INFO : Loading a fresh vocabulary
2017-05-16 12:31:58,830 : INFO : min_count=5 retains 3346 unique words (29% of original 11431, drops 8085)
2017-05-16 12:31:58,832 : INFO : min_count=5 leaves 175425 word corpus (93% of original 188505, drops 13080)
2017-05-16 12:31:58,840 : INFO : deleting the raw counts dictionary of 11431 items
2017-05-16 12:31:58,841 : INFO : sample=0.001 downsamples 41 most-common words
2017-05-16 12:31:58,842 : INFO : downsampling leaves estimated 125739 word corpus (71.7% of prior 175425)
2017-05-16 12:31:58,843 : INFO : estimated required memory for 3346 words and 300 dimensions: 9703400 bytes
2017-05-16 12:31:58,852 : INFO : resetting layer weights
2017-05-

Parsing datasets sentences
Training Word2Vec Model


2017-05-16 12:31:59,409 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-05-16 12:31:59,414 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-05-16 12:31:59,418 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-05-16 12:31:59,420 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-05-16 12:31:59,421 : INFO : training on 942525 raw words (628611 effective words) took 0.5s, 1211803 effective words/s
2017-05-16 12:31:59,421 : INFO : precomputing L2-norms of word weight vectors
2017-05-16 12:31:59,439 : INFO : saving Word2Vec object under ../../Results/Yeast/yeast_strict_model, separately None
2017-05-16 12:31:59,440 : INFO : not storing attribute syn0norm
2017-05-16 12:31:59,440 : INFO : not storing attribute cum_table
2017-05-16 12:31:59,513 : INFO : saved ../../Results/Yeast/yeast_strict_model


Parsing datasets sentences


2017-05-16 12:32:00,025 : INFO : collecting all words and their counts
2017-05-16 12:32:00,026 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-16 12:32:00,070 : INFO : PROGRESS: at sentence #10000, processed 243957 words, keeping 11334 word types
2017-05-16 12:32:00,118 : INFO : PROGRESS: at sentence #20000, processed 486019 words, keeping 14899 word types
2017-05-16 12:32:00,163 : INFO : PROGRESS: at sentence #30000, processed 729058 words, keeping 17564 word types
2017-05-16 12:32:00,207 : INFO : PROGRESS: at sentence #40000, processed 970773 words, keeping 19761 word types


...

2017-05-16 12:32:00,242 : INFO : collected 21147 word types from a corpus of 1155938 raw words and 47662 sentences
2017-05-16 12:32:00,243 : INFO : Loading a fresh vocabulary
2017-05-16 12:32:00,266 : INFO : min_count=5 retains 8077 unique words (38% of original 21147, drops 13070)
2017-05-16 12:32:00,266 : INFO : min_count=5 leaves 1134115 word corpus (98% of original 1155938, drops 21823)
2017-05-16 12:32:00,288 : INFO : deleting the raw counts dictionary of 21147 items
2017-05-16 12:32:00,290 : INFO : sample=0.001 downsamples 46 most-common words
2017-05-16 12:32:00,290 : INFO : downsampling leaves estimated 829411 word corpus (73.1% of prior 1134115)
2017-05-16 12:32:00,291 : INFO : estimated required memory for 8077 words and 300 dimensions: 23423300 bytes
2017-05-16 12:32:00,319 : INFO : resetting layer weights
2017-05-16 12:32:00,411 : INFO : training model with 4 workers on 8077 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=6
2017-05-16 12:32:00,41

Parsing datasets sentences


2017-05-16 12:32:07,052 : INFO : collecting all words and their counts
2017-05-16 12:32:07,053 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-16 12:32:07,100 : INFO : PROGRESS: at sentence #10000, processed 229844 words, keeping 12368 word types
2017-05-16 12:32:07,150 : INFO : PROGRESS: at sentence #20000, processed 458659 words, keeping 16803 word types
2017-05-16 12:32:07,198 : INFO : PROGRESS: at sentence #30000, processed 687887 words, keeping 20163 word types
2017-05-16 12:32:07,251 : INFO : PROGRESS: at sentence #40000, processed 916854 words, keeping 22943 word types


Training Word2Vec Model


2017-05-16 12:32:07,302 : INFO : PROGRESS: at sentence #50000, processed 1145367 words, keeping 25239 word types
2017-05-16 12:32:07,353 : INFO : PROGRESS: at sentence #60000, processed 1372926 words, keeping 27354 word types
2017-05-16 12:32:07,393 : INFO : PROGRESS: at sentence #70000, processed 1602439 words, keeping 29086 word types
2017-05-16 12:32:07,436 : INFO : PROGRESS: at sentence #80000, processed 1832259 words, keeping 30779 word types
2017-05-16 12:32:07,479 : INFO : PROGRESS: at sentence #90000, processed 2061578 words, keeping 32268 word types
2017-05-16 12:32:07,522 : INFO : PROGRESS: at sentence #100000, processed 2291831 words, keeping 33740 word types
2017-05-16 12:32:07,571 : INFO : PROGRESS: at sentence #110000, processed 2520309 words, keeping 35115 word types
2017-05-16 12:32:07,611 : INFO : PROGRESS: at sentence #120000, processed 2749954 words, keeping 36217 word types
2017-05-16 12:32:07,654 : INFO : PROGRESS: at sentence #130000, processed 2977684 words, keep

In [24]:
for seed in random_seeds:
    data_name = '../../Results/Yeast/train_test/yeast_tr_te_split_' + str(seed)
    train_data = pickle.load(open(data_name + '_train_data.pkl', 'rb'))
    train_labels = pickle.load(open(data_name + '_train_labels.pkl', 'rb'))
    test_data = pickle.load(open(data_name + '_test_data.pkl', 'rb'))
    test_labels = pickle.load(open(data_name + '_test_labels.pkl', 'rb'))

    w2v_train_vecs, w2v_test_vecs = pred.word_2_vec_feat_vecs(train_data, test_data, yeast_w2v_model_strict, feature_count=300)

    strict_list_SR_dims_param = [w2v_train_vecs, w2v_test_vecs,
                                 train_labels, test_labels]

    w2v_train_vecs, w2v_test_vecs = pred.word_2_vec_feat_vecs(train_data, test_data, yeast_w2v_model_gen, feature_count=300)

    strict_list_GEN_dims_param = [w2v_train_vecs, w2v_test_vecs,
                                  train_labels, test_labels]

    w2v_train_vecs, w2v_test_vecs = pred.word_2_vec_feat_vecs(train_data, test_data, yeast_w2v_model_be, feature_count=300)

    strict_list_BE_dims_param = [w2v_train_vecs, w2v_test_vecs,
                                 train_labels, test_labels]

    pickle.dump(strict_list_SR_dims_param, open('../../Results/Yeast/result_list/yeast_strict_list_SR_'+str(seed)+'_results_list.pkl', 'wb'))
    pickle.dump(strict_list_GEN_dims_param, open('../../Results/Yeast/result_list/yeast_strict_list_GEN_'+str(seed)+'_results_list.pkl', 'wb'))
    pickle.dump(strict_list_BE_dims_param, open('../../Results/Yeast/result_list/yeast_strict_list_BE_'+str(seed)+'_results_list.pkl', 'wb'))

In [7]:
def modelfit(alg, train_vecs, train_labels, w2v_model_type, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param=alg.get_xgb_params()
        xgtrain = xgb.DMatrix(train_vecs, 
                              label=train_labels)
        cvresult = xgb.cv(xgb_param, 
                          xgtrain, 
                          num_boost_round=alg.get_params()['n_estimators'], 
                          nfold=cv_folds, 
                          metrics='auc', 
                          early_stopping_rounds=50)
        alg.set_params(n_estimators=cvresult.shape[0])
        
    #fit the algorithm on the data
    alg.fit(train_vecs, train_labels, eval_metric='auc')
    
    #Predict training set:
    train_predictions = alg.predict(train_vecs)
    train_predprob = alg.predict_proba(train_vecs)[:,1]
    
    #Print Model report:
    print(w2v_model_type, '\nModel Report')
    print(w2v_model_type, 'Accuracy: %.4g' % metrics.accuracy_score(train_labels, train_predictions))
    print(w2v_model_type, 'AUC Score (Train): %f' % metrics.roc_auc_score(train_labels, train_predprob))
    
#     feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
#     feat_imp.plot(kind='bar', title='Feature Importance')
#     plt.ylabel('Feature Importance Score')

    error = 1-metrics.accuracy_score(train_labels, train_predictions)
    auc = metrics.roc_auc_score(train_labels, train_predprob)
    
    return error, auc

In [18]:
xgb1 = XGBClassifier(learning_rate = 0.1, 
                     n_estimators=1000, 
                     max_depth=5, 
                     min_child_weight=1, 
                     gamma=0, 
                     subsample=0.8, 
                     colsample_bytree=0.8, 
                     objective='binary:logistic', 
                     nthread=4, 
                     scale_pos_weight=1, 
                     seed=24)

# strict_error_list = []
# strict_auc_list = []
# gen_error_list = []
# gen_auc_list = []
# be_error_list = []
# be_auc_list = []

seed = 144
  
strict_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_SR_dims_param_'+str(seed)+'_results_list.pkl', 'rb'))
gen_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_GEN_dims_param_'+str(seed)+'_results_list.pkl', 'rb'))
be_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_BE_dims_param_'+str(seed)+'_results_list.pkl', 'rb'))
print(seed)
strict_error, strict_auc = modelfit(xgb1, strict_list[0], strict_list[2], 'STRICT')
gen_error, gen_auc = modelfit(xgb1, gen_list[0], gen_list[2], 'GEN')
be_error, be_auc = modelfit(xgb1, be_list[0], be_list[2], 'BE')

# strict_error_list.append(strict_error)
# strict_auc_list.append(strict_auc)
# gen_error_list.append(gen_error)
# gen_auc_list.append(gen_auc)
# be_error_list.append(be_error)
# be_auc_list.append(be_auc)

144
STRICT 
Model Report
STRICT Accuracy: 0.6019
STRICT AUC Score (Train): 0.628022
GEN 
Model Report
GEN Accuracy: 0.6309
GEN AUC Score (Train): 0.672836
BE 
Model Report
BE Accuracy: 0.6039
BE AUC Score (Train): 0.654637


In [20]:
xgb1 = XGBClassifier(learning_rate = 0.1, 
                     n_estimators=1000, 
                     max_depth=5, 
                     min_child_weight=1, 
                     gamma=0, 
                     subsample=0.8, 
                     colsample_bytree=0.8, 
                     objective='binary:logistic', 
                     nthread=4, 
                     scale_pos_weight=1, 
                     seed=24)

# strict_error_list = []
# strict_auc_list = []
# gen_error_list = []
# gen_auc_list = []
# be_error_list = []
# be_auc_list = []

seed = 144
  
strict_list_old = pickle.load(open('../../Results/Yeast/old_results/yeast_strict_real_pure_144no_BOW_results_list.pkl', 'rb'))
gen_list_old = pickle.load(open('../../Results/Yeast/old_results/yeast_strict_real_gen_mod_144no_BOW_results_list.pkl', 'rb'))
be_list_old = pickle.load(open('../../Results/Yeast/old_results/yeast_strict_real_be_mod_144no_BOW_results_list.pkl', 'rb'))
print(seed)
strict_error, strict_auc = modelfit(xgb1, strict_list[0], strict_list[2], 'STRICT')
gen_error, gen_auc = modelfit(xgb1, gen_list[0], gen_list[2], 'GEN')
be_error, be_auc = modelfit(xgb1, be_list[0], be_list[2], 'BE')

144
STRICT 
Model Report
STRICT Accuracy: 0.7953
STRICT AUC Score (Train): 0.874488
GEN 
Model Report
GEN Accuracy: 0.8233
GEN AUC Score (Train): 0.905831
BE 
Model Report
BE Accuracy: 0.8428
BE AUC Score (Train): 0.918466


In [25]:
xgb1 = XGBClassifier(learning_rate = 0.1, 
                     n_estimators=1000, 
                     max_depth=5, 
                     min_child_weight=1, 
                     gamma=0, 
                     subsample=0.8, 
                     colsample_bytree=0.8, 
                     objective='binary:logistic', 
                     nthread=4, 
                     scale_pos_weight=1, 
                     seed=24)

# strict_error_list = []
# strict_auc_list = []
# gen_error_list = []
# gen_auc_list = []
# be_error_list = []
# be_auc_list = []

seed = 144
  
strict_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_SR_'+str(seed)+'_results_list.pkl', 'rb'))
gen_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_GEN_'+str(seed)+'_results_list.pkl', 'rb'))
be_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_BE_'+str(seed)+'_results_list.pkl', 'rb'))
print(seed)
strict_error, strict_auc = modelfit(xgb1, strict_list[0], strict_list[2], 'STRICT')
gen_error, gen_auc = modelfit(xgb1, gen_list[0], gen_list[2], 'GEN')
be_error, be_auc = modelfit(xgb1, be_list[0], be_list[2], 'BE')

# strict_error_list.append(strict_error)
# strict_auc_list.append(strict_auc)
# gen_error_list.append(gen_error)
# gen_auc_list.append(gen_auc)
# be_error_list.append(be_error)
# be_auc_list.append(be_auc)

144
STRICT 
Model Report
STRICT Accuracy: 0.9327
STRICT AUC Score (Train): 0.990888
GEN 
Model Report
GEN Accuracy: 0.6287
GEN AUC Score (Train): 0.684977
BE 
Model Report
BE Accuracy: 0.654
BE AUC Score (Train): 0.724163


In [21]:
strict_list[2] == strict_list_old[2]

True

In [32]:
print('Strict Results Error: ', np.mean(strict_error_list), ', Auc: ', np.mean(strict_auc_list))
print('Gen Results Error: ', np.mean(gen_error_list), ', Auc: ', np.mean(gen_auc_list))
print('Be Results Error: ', np.mean(be_error_list), ', Auc: ', np.mean(be_auc_list))

Strict Results Error:  0.378943362667 , Auc:  0.66059997443
Gen Results Error:  0.371137665262 , Auc:  0.669424603342
Be Results Error:  0.381628285206 , Auc:  0.656050922499


In [36]:
param_test1 = {
    'max_depth':[3,4,5,6,7,8,9,10],
    'min_child_weight':[1,2,3,4,5,6]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, 
                                                  n_estimators=140, 
                                                  max_depth=5, 
                                                  min_child_weight=1, 
                                                  gamma=0, 
                                                  subsample=0.8, 
                                                  colsample_bytree=0.8, 
                                                  objective='binary:logistic', 
                                                  nthread=4, 
                                                  scale_pos_weight=1, 
                                                  seed=24), 
                        param_grid=param_test1, 
                        scoring='roc_auc', 
                        n_jobs=4, 
                        iid=False, 
                        cv=5)

In [37]:
strict_param_test1 = []
for seed in random_seeds:
    strict_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_SR_dims_param_'+str(seed)+'_results_list.pkl', 'rb'))
    gsearch1.fit(strict_list[0], strict_list[2])
    strict_param_test1.append((seed, (gsearch1.best_params_, gsearch1.best_score_)))

KeyboardInterrupt: 

In [ ]:
gen_param_test1 = []
for seed in random_seeds:
    gen_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_GEN_dims_param_'+str(seed)+'_results_list.pkl', 'rb'))
    gsearch1.fit(gen_list[0], gen_list[2])
    gen_param_test1.append((seed, (gsearch1.best_params_, gsearch1.best_score_)))

In [ ]:
be_param_test1 = []
for seed in random_seeds:
    be_list = pickle.load(open('../../Results/Yeast/result_list/yeast_strict_list_BE_dims_param_'+str(seed)+'_results_list.pkl', 'rb'))
    gsearch1.fit(be_list[0], be_list[2])
    be_param_test1.append((seed, (gsearch1.best_params_, gsearch1.best_score_)))

In [ ]:
for i in range(10):
    print('\n', strict_param_test1[i], '\n', gen_param_test1[i], '\n', be_param_test1[i], '\n')